<a href="https://colab.research.google.com/github/Phionanamugga/Autonomous_Navigation/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2
import torch
from ultralytics import YOLO
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
import time
import supervision as sv

In [3]:
# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # Pre-trained YOLOv8 nano model

In [4]:
# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, nn_budget=100, nms_max_overlap=1.0)

In [5]:
# Initialize video capture (replace with your video file or camera)
cap = cv2.VideoCapture('sample_traffic_video.mp4')

In [7]:
# Initialize supervision annotators
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()


In [8]:

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    results = model(frame)[0]
    detections = []
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf)
        cls = int(box.cls)
        if conf > 0.5:  # Confidence threshold
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls))

    # Update tracker with detections
    tracks = tracker.update_tracks(detections, frame=frame)

    # Prepare detections for supervision
    bboxes = []
    class_ids = []
    confidences = []
    track_ids = []

    for track in tracks:
        if not track.is_confirmed():
            continue
        ltrb = track.to_ltrb()
        bboxes.append(ltrb)
        class_ids.append(track.det_class)
        confidences.append(track.det_conf)
        track_ids.append(track.track_id)

    # Convert to supervision Detections
    detections = sv.Detections(
        xyxy=np.array(bboxes),
        class_id=np.array(class_ids),
        confidence=np.array(confidences),
        tracker_id=np.array(track_ids)
    )

    # Annotate frame
    annotated_frame = box_annotator.annotate(scene=frame.copy(), detections=detections)
    labels = [f"ID {track_id} {model.names[class_id]}" for track_id, class_id in zip(detections.tracker_id, detections.class_id)]
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

    # Display frame
    cv2.imshow('Object Detection and Tracking', annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
